In [280]:
import numpy as np
import time

In [166]:
pieces = {
    'A': {'shape': np.array([
        [1, 0, 0],
        [1, 1, 1],
        [0, 1, 0],
        [0, 1, 0]
    ]) * 2, 'val': 2},
    'B': {'shape': np.array([
        [1, 1, 0],
        [0, 1, 0],
        [0, 1, 1],
        [0, 1, 0]
    ]) * 3, 'val': 3},
    'C': {'shape': np.array([
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 1],
        [0, 0, 1]
    ]) * 4, 'val': 4},
    'D': {'shape': np.array([
        [1, 1, 0],
        [0, 1, 1],
        [0, 1, 0],
        [0, 1, 0]
    ]) * 5, 'val': 5},
    'E': {'shape': np.array([
        [0, 1, 0],
        [1, 1, 1],
        [0, 1, 0],
        [0, 1, 0]
    ]) * 6, 'val': 6},
    'F': {'shape': np.array([
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 1],
        [0, 1, 0]
    ]) * 7, 'val': 7},
    'G': {'shape': np.array([
        [0, 1, 0],
        [1, 1, 0],
        [0, 1, 1],
        [0, 1, 0]
    ]) * 8, 'val': 8},
    'H': {'shape': np.array([
        [1, 0],
        [1, 0],
        [1, 1],
        [0, 1],
        [0, 1]
    ]) * 9, 'val': 9},
    'I': {'shape': np.array([
        [0, 1, 0],
        [1, 1, 1],
        [0, 0, 1],
        [0, 0, 1]
    ]) * 10, 'val': 10}
    
}

In [235]:
class CannotPlacePieceError(Exception):
    pass

In [243]:
class Board:
    def __init__(self, l = 11, w = 11):
        self.l = l
        self.w = w
        self.board = np.ones((self.l+2, self.w+2))
        self.board[1:-1,1:-1] = np.zeros((self.l, self.w))
        self.board = self.board.astype(int)
#         self.board[1:12, 2:11] = 1
#         self.board[2:11, 1:12] = 1
#         self.board[4:9, 4:9] = 0
        self.pieces = []
        
        
    def place_piece(self, piece, x, y, flipped, orientation):
        piece.place(x, y, flipped, orientation)
        if self.verify_valid_placement(piece):
            self.pieces.append(piece)
            self.board[1:-1,1:-1] += piece.board_pos
            self.block_positions()
        else:
            raise CannotPlacePieceError(f'Cannot place {piece}')
        
        
    def verify_valid_placement(self, piece):
        return np.sum(self.board[1:-1,1:-1] * piece.board_pos) == 0
            
            
    def block_positions(self):
        for i in range(1, 12):
            for j in range(1, 12):
                if self.board[i, j] == 0:
                    if np.sum(self.board[i-1:i+2,j-1:j+2] > 1) > 0:
                        self.board[i, j] = 1
                
        
    def __str__(self):
        row_div = ' _____' * (self.w + 2) + ' \n'
        col_div = '      ' + '|     ' * self.w + '|     ' + '\n'
        end_col_div = '      ' + '|     ' * self.w + '|     ' + '\n'
        _div = '      ' * (self.w + 2) + '\n'
        table = end_col_div + (row_div + _div + col_div) * self.l + row_div + _div + end_col_div
        for piece in self.pieces:
            for i in range(self.l+2):
                for j in range(self.w+2):
                    pos = 238*(i) + 6*(j) + 3 # self.w = 11
                    # Changes the 0 to the right number
                    if self.board[i,j] == piece.val:
                        table = table[:pos] + piece.label + table[pos+1:]
                        # deletes lines below
                        pos_ = pos + 79
                        table = table[:pos_-2] + '     ' + table[pos_+3:]
                        # deletes lines above
                        pos_ = pos - 159
                        table = table[:pos_-2] + '     ' + table[pos_+3:]
                        # deletes lines left
                        table = table[:pos-3] + '   ' + table[pos:]
                        # deletes lines right
                        table = table[:pos+1] + '   ' + table[pos+4:]

                    if self.board[i, j] == 1:
                        try:
                            top = self.board[i-1,j] == piece.val
                        except:
                            top = False
                        try:
                            bottom = self.board[i+1,j] == piece.val
                        except:
                            bottom = False
                        try:
                            left = self.board[i,j-1] == piece.val
                        except:
                            left = False
                        try:
                            right = self.board[i,j+1] == piece.val
                        except:
                            right = False
                        try:
                            top_l = self.board[i-1, j-1] == piece.val
                        except:
                            top_l = False
                        try:
                            top_r = self.board[i-1, j+1] == piece.val
                        except:
                            top_r = False
                        try:
                            bottom_l = self.board[i+1, j-1] == piece.val
                        except:
                            bottom_l = False
                        try:
                            bottom_r = self.board[i+1, j+1] == piece.val
                        except:
                            bottom_r = False
                        # outside corners
                        if not top and not bottom and not left and not right:
                            # top left corner
                            if bottom_r:
                                table = table[:pos+1] + ' _' + table[pos+3:]
                                pos_ = pos + 79
                                table = table[:pos_+1] + '| ' + table[pos_+3:]

                            # top right corner
                            if bottom_l:
                                table = table[:pos-2] + '_' + table[pos-1:]
                                pos_ = pos + 79
                                table = table[:pos_-2] + ' |' + table[pos_:]

                            # bottom left corner
                            if top_r:
                                pos_ = pos - 78
                                table = table[:pos_] + '|_' + table[pos_+2:]

                            # bottom right corner
                            if top_l:
                                pos_ = pos - 78
                                table = table[:pos_-3] + '_|' + table[pos_-1:]

                        # top
                        if not top and bottom and not left and not right:
                            table = table[:pos-2] + '_____' + table[pos+3:]
                            pos_ = pos + 79
                            table = table[:pos_-2] + '     ' + table[pos_+3:]

                        # bottom
                        if top and not bottom and not left and not right:
                            pos_ = pos - 79
                            table = table[:pos_-2] + '_____' + table[pos_+3:]
                            pos_ = pos - 159
                            table = table[:pos_-2] + '     ' + table[pos_+3:]

                        # left
                        if not top and not bottom and not left and right:
                            table = table[:pos+1] + '|  ' + table[pos+4:]
                            pos_ = pos + 80
                            table = table[:pos_] + '| ' + table[pos_+2:]
                            pos_ = pos - 78
                            table = table[:pos_] + '|' + table[pos_+1:]

                        # right
                        if not top and not bottom and left and not right:
                            table = table[:pos-3] + '  |' + table[pos:]
                            pos_ = pos + 78
                            table = table[:pos_-1] + ' |' + table[pos_+1:]
                            pos_ = pos - 80
                            table = table[:pos_] + '|' + table[pos_+1:]

                        # top left inside corner
                        if top and left:
                            table = table[:pos-1] + '|' + table[pos:]
                            pos_ = pos + 78
                            table = table[:pos_-2] + '  |' + table[pos_+1:]
                            pos_ = pos - 80
                            table = table[:pos_+1] + '___' + table[pos_+4:]

                        # top right inside corner
                        if top and right:
                            table = table[:pos+1] + '|' + table[pos+2:]
                            pos_ = pos + 78
                            table = table[:pos_+2] + '| ' + table[pos_+4:]
                            pos_ = pos - 80
                            table = table[:pos_-1] + '___' + table[pos_+2:]

                        # bottom left inside corner
                        if bottom and left:
                            table = table[:pos-1] + '|___' + table[pos+3:]
                            pos_ = pos - 80
                            table = table[:pos_] + '|' + table[pos_+1:]

                        # bottom right inside corner
                        if bottom and right:
                            table = table[:pos-2] + '___|' + table[pos+2:]
                            pos_ = pos - 80
                            table = table[:pos_+2] + '|' + table[pos_+3:]

        return table

In [244]:
class Piece:
    def __init__(self, label):
        self.label = label
        self.shape = pieces[label]['shape']
        self.val = pieces[label]['val']
        self.h = self.shape.shape[0]
        self.w = self.shape.shape[1]
        self.x = 0
        self.y = 0
        self.flipped = False
        self.orientation = 'n'
        self.board_h = self.h
        self.board_w = self.w
        self.on_board = False
        
        board = np.zeros((11, 11))
        board[self.y:self.y+self.board_h, self.x:self.x+self.board_w] = self.shape
        self.board_pos = board.astype(int)
        
        
    def flip_and_rotate(self):
        if self.orientation == 'n':
            return self.shape[:,::-1] if self.flipped else self.shape
        elif self.orientation == 'e':
            return self.shape[::-1,::-1].T if self.flipped else self.shape[::-1,:].T
        elif self.orientation == 's':
            return self.shape[::-1,:] if self.flipped else self.shape[::-1,::-1]
        elif self.orientation == 'w':
            return self.shape.T if self.flipped else self.shape[:,::-1].T
            
        
    def update_board_pos(self):
        shape_transformed = self.flip_and_rotate()
        board = np.zeros((11, 11))
        board[self.y:self.y+self.board_h, self.x:self.x+self.board_w] = shape_transformed
        self.board_pos = board.astype(int)
        
    
    def place(self, x, y, flipped, orientation):
        self.x = x
        self.y = y
        self.flipped = flipped
        self.orientation = orientation
        if orientation in ['e', 'w']:
            self.board_h = self.w
            self.board_w = self.h
        self.update_board_pos()
        
        
    def __repr__(self):
        return self.label
        

In [245]:
board = Board()
board.place_piece(Piece("A"), 0, 0, 1, 's')
board.place_piece(Piece("B"), 3, 0, 0, 'n')
board.place_piece(Piece("E"), 5, 4, 0, 'n')
board.place_piece(Piece("I"), 3, 0, 0, 'n')
print(board)

CannotPlacePieceError: Cannot place I

In [270]:
Pieces = [Piece(piece) for piece in pieces]
board = Board()
failed = False
while len(board.pieces) < 8: 
    for piece in Pieces:
        x = 0
        y = 0
        flipped = False
        orientation = 'n'
        placed = False
        while piece not in board.pieces:
            try:
                board.place_piece(piece, x, y, flipped, orientation)
                placed = True
            except:
                x += 1
                if x == 10:
                    x = 0
                    y += 1
                    if y == 10:
                        y = 0
                        flipped = not flipped
                        if not flipped:
                            if orientation == 'n':
                                orientation = 'e'
                            elif orientation == 'e':
                                orientation = 's'
                            elif orientation == 's':
                                orientation = 'w'
                            else:
                                failed = True
                                break
        if placed or failed:
            break
    if failed:
        break
if failed:
    print(f'{"-"*6*13}\n{" "*28}Failed to place piece {piece}\n{"-"*6*13}\n\n\n')
print(board)

------------------------------------------------------------------------------
                            Failed to place piece H
------------------------------------------------------------------------------



     _|_____|_    |     |    _|_____|_____|_    |    _|_____|_    |     |     
 ___|         |___ _____ ___|               |___ ___|         |___ _____ _____ 
    |         |             |               |       |         |               
    |    A    |___|_____|_  |    B     B    |   |   |    C    |___|_    |     
 ___|                     |_|               |___ ___|               |___ _____ 
    |                     | |_ ___          |       |               |         
    |    A     A     A    |   |   |    B    |___|_  |    C     C    |___|_    
 ___|                     |___ ___|               |_|                     |___ 
    |_ ___           ___ _|       |               | |_ ___                |   
      |   |    A    |   |     |   |    B     B    |   |   |    C     C   

In [272]:
(10*10*2*4)**9

134217728000000000000000000

{A: {'x': 0, 'y': 0, 'flipped': False, 'orientation': 'n'},
 B: {'x': 0, 'y': 0, 'flipped': False, 'orientation': 'n'},
 C: {'x': 0, 'y': 0, 'flipped': False, 'orientation': 'n'},
 D: {'x': 0, 'y': 0, 'flipped': False, 'orientation': 'n'},
 E: {'x': 0, 'y': 0, 'flipped': False, 'orientation': 'n'},
 F: {'x': 0, 'y': 0, 'flipped': False, 'orientation': 'n'},
 G: {'x': 0, 'y': 0, 'flipped': False, 'orientation': 'n'},
 H: {'x': 0, 'y': 0, 'flipped': False, 'orientation': 'n'},
 I: {'x': 0, 'y': 0, 'flipped': False, 'orientation': 'n'}}

In [282]:
Pieces = [Piece(piece) for piece in pieces]
state = {'x': 0, 'y': 0, 'flipped': False, 'orientation': 'n'}
state_dict = {piece: state for piece in Pieces}
state_dict
i = 0
board = Board()
start = time.time()
for x in range(10):
    state_dict[Pieces[0]]['x'] = x
    for y in range(10):
        state_dict[Pieces[0]]['y'] = y
        for flipped in [True, False]:
            state_dict[Pieces[0]]['flipped'] = flipped
            for orientation in ['n', 'e', 's', 'w']:
                state_dict[Pieces[0]]['orientation'] = orientation
                try:
                    i += 1
                    for piece in Pieces:
                        board.place_piece(piece, state_dict[piece][x], state_dict[piece][y], state_dict[piece][flipped], state_dict[piece][orientation])
                except:
                    continue
end = time.time()
end - start, i

0.0022928714752197266


(None, 800)

In [295]:
num_years = ((72*8)**8)*70*8 * (end - start) / 800 / 60 / 60 / 24 / 365
"{:,}".format(int(num_years)) + ' years'

'616,667,430,900 years'